In [14]:
import pandas as pd
import numpy as np

In [15]:
input_files = ['negotiations/human_bot_round1.csv', 
               'negotiations/human_bot_round2.csv', 
               'negotiations/human_bot_round3.csv', 
               'negotiations/human_bot_round4.csv']
output_file = 'outcomes/human_bot_outcomes.csv'

# Initialize an empty DataFrame to store combined data
df = pd.DataFrame()

# Iterate through each file name
for data_file in input_files:
    # Read into a pd dataframe or combine them
    df = pd.concat([df, pd.read_csv(data_file, delimiter=',')], ignore_index=True)

df

,firstname,lastname,email,attempt,group,role,privateSurvey14.player.iQ1,url1.player.oShow,privateSurvey16.player.iNewIssue1,privateSurvey16.player.iNewIssue2,...,Unnamed: 228,Unnamed: 229,Unnamed: 230,Unnamed: 231,Unnamed: 232,Unnamed: 233,Unnamed: 234,Unnamed: 235,Unnamed: 236,Unnamed: 237
0,Steffen,Wenzel-Halangk,25steffen@web.de,0,45,1,I agree to participate,0.0,Male,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Steffen,Wenzel-Halangk,25steffen@web.de,1,47,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Steffen,Wenzel-Halangk,25steffen@web.de,2,49,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Steffen,Wenzel-Halangk,25steffen@web.de,3,50,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Steffen,Wenzel-Halangk,25steffen@web.de,4,51,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4126,Edda,Laster,NaN,0,341,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4127,Zheny,Avramova,z.zhelyazkova@gmail.com,0,342,2,I agree to participate,0,Female,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4128,Annette,Barker,NaN,0,342,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4129,Jared,Curhan,curhan@mit.edu,0,343,1,I agree to participate,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Calculate subjective value from the bot's perspective (reverse scoring already applied)
fields = [f'privateSurvey3.player.oSVI{i}NCounterpart' for i in range(1, 17)]
df[fields] = df[fields].apply(pd.to_numeric, errors='coerce')
df['svi_bot'] = df[fields].mean(axis=1)

# Calculate subjective value from the human's perspective (reverse scoring already applied)
# Irrelevant for human-bot negotiations
fields = [f'privateSurvey3.player.oSVI{i}N' for i in range(1, 17)]
df[fields] = df[fields].apply(pd.to_numeric, errors='coerce')
df['svi_human'] = df[fields].mean(axis=1)

In [ ]:
# Clean-up data frame
# role = the role of the human in the negotiation, 1-Consultant, 2-COO
# group = negotiation id
col_names = ['group', 'prompt_author_email', 'prompt_name', 'prompt', 'human_email', 
            'attempt', 'role', 'deal_reached', 'value_created', 'transcript', 'points_human', 'points_bot',
            'svi_human', 'svi_bot'
            ]

# Rename columns in the DataFrame as per the instructions
df['deal_reached'] = df['publicSurvey1.group.oAgreementHelper']
df['points_human'] = df['publicSurvey1.player.oGains']
df['points_bot'] = df['publicSurvey1.group.oGainsRole0Group']
df['value_created'] = df['publicSurvey1.group.oTotals']
df['transcript'] = df['publicSurvey1.chatNegotiationHistoryRaw']
df['prompt_name'] = df['publicSurvey1.promptVariationName']
df['prompt'] = df['publicSurvey1.promptVariation']
df['prompt_author_email'] = df['publicSurvey1.promptVariationAuthorEmail']
df['human_email'] = df['email']

df_cleaned = df[col_names]
df_cleaned = df_cleaned[(df_cleaned['transcript'].notna()) & df_cleaned['deal_reached'].notna()] # filter to completed negotiations
df_cleaned = df_cleaned[df_cleaned['role'].isin(['1', '2'])]
df_cleaned

,group,prompt_author_email,prompt_name,prompt,human_email,attempt,role,deal_reached,value_created,transcript,points_human,points_bot,svi_human,svi_bot
0,45,sidratulhayat507160@gmail.com,Apex,"In today's complex world, negotiations occur i...",25steffen@web.de,0,1,True,4200.0,[COO]: Thank you for meeting with me today. I ...,1450.0,1450.0,3.937500,4.125
1,47,oscar@egixia.com,NEXUS NEGOTIATOR 1.0,General Objective: Act as a cutting-edge negot...,25steffen@web.de,1,1,True,4400.0,[Consultant]: Hello [COO]: Hello! Thank you fo...,1550.0,1550.0,5.187500,4.000
2,49,chentouf@mit.ed,Stata,"You are an expert negotiator, who is dominant,...",25steffen@web.de,2,1,True,2900.0,[COO]: Hello! I'm looking forward to our discu...,1200.0,1200.0,3.750000,4.000
3,50,a.schrader@andreas-schrader.com,ASC Final,"ROLE: You are ASC, a Elite Negotiation AI comb...",25steffen@web.de,3,1,True,4200.0,[Consultant]: Hello [COO]: Hello! Thank you fo...,1950.0,1950.0,5.750000,4.000
4,51,dawnmarchand@helenistic.com,EcoFashion,A global luxury fashion brand is negotiating a...,25steffen@web.de,4,1,True,4300.0,[COO]: Thank you for meeting with me today. I’...,1900.0,1900.0,5.500000,4.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4115,336,andriy.basov@gmail.com,Determinator,Negotiation Agent Prompt Role: You are a highl...,douglas.kiang@gmail.com,5,1,TRUE,4200,[Consultant]: Hi [COO]: Hello! It's great to m...,2750,2750.0,5.187500,4.000
4120,338,kenku8@gmail.com,Negoken_9,System Prompt � Master Negotiator Mode You are...,constanze.reichetzer@unisg.ch,1,2,TRUE,4400,"[COO]: Hi, I'm very much looking forward to yo...",3000,1400.0,5.437500,4.000
4122,339,alvaro.valencia@gmail.com,AVBotTurboCharged,Your overarching goal is to reach a deal and t...,constanze.reichetzer@unisg.ch,2,2,TRUE,4300,"[COO]: Hi, I'm very happy to hear that our CEO...",3200,1100.0,4.133333,4.000
4124,340,samphadnis@gmail.com,Fair Negotiator,I want you to act as a skilled negotiator. I w...,constanze.reichetzer@unisg.ch,3,2,TRUE,4500,"[COO]: Hi, I'm very much looking forward to wo...",3300,1200.0,3.625000,4.000


In [ ]:
# only include one row per negotiation, keeping it from the human's perspective


In [13]:
# save df_cleaned to csv
df_cleaned.to_csv(output_file, index=False)